Data: https://www.kaggle.com/mustfkeskin/turkish-movie-sentiment-analysis-dataset/code

In [361]:
# Check the GPU colab gave to us.
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-cf47de9d-cec0-d1b1-d788-3b3a4288e3a7)


### Preprocess data

In [362]:
# Get data
import pandas as pd

df = pd.read_csv("magaza_yorumlari_duygu_analizi.csv", encoding="utf-16")
df.head()

,Görüş,Durum
0,"ses kalitesi ve ergonomisi rezalet, sony olduğ...",Olumsuz
1,hizli teslimat tesekkürler,Tarafsız
2,ses olayı süper....gece çalıştır sıkıntı yok.....,Olumlu
3,geldi bigün kullandık hemen bozoldu hiçtavsiye...,Olumsuz
4,Kulaklığın sesi kaliteli falan değil. Aleti öv...,Olumsuz


In [363]:
# Check the DataFrame to see the number of lines and non-null objects
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11429 entries, 0 to 11428
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Görüş   11426 non-null  object
 1   Durum   11429 non-null  object
dtypes: object(2)
memory usage: 178.7+ KB


In [364]:
# Check value counts to see whether the data is balanced or not
df.Durum.value_counts()

Olumlu      4253
Olumsuz     4238
Tarafsız    2938
Name: Durum, dtype: int64

In [365]:
# Since some nulls might be seen as a float, drop na to not face any problems.
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11426 entries, 0 to 11428
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Görüş   11426 non-null  object
 1   Durum   11426 non-null  object
dtypes: object(2)
memory usage: 267.8+ KB


In [366]:
# Check the first line in df.Görüş
df.Görüş[0]

"ses kalitesi ve ergonomisi rezalet, sony olduğu için aldım ama 4'de 1 fiyatına çin replika ürün alsaydım çok çok daha iyiydi, kesinlikle tavsiye etmiyorum."

In [367]:
# Remove punctuation for our model to learn better
df['Görüş'] = df['Görüş'].str.replace(r'[^\w\s]+', '')
df.Görüş[0]

'ses kalitesi ve ergonomisi rezalet sony olduğu için aldım ama 4de 1 fiyatına çin replika ürün alsaydım çok çok daha iyiydi kesinlikle tavsiye etmiyorum'

In [368]:
# Lower the inputs for our model to learn better
df["Görüş"] = df["Görüş"].str.lower()
df.head()

,Görüş,Durum
0,ses kalitesi ve ergonomisi rezalet sony olduğu...,Olumsuz
1,hizli teslimat tesekkürler,Tarafsız
2,ses olayı süpergece çalıştır sıkıntı yokkablo ...,Olumlu
3,geldi bigün kullandık hemen bozoldu hiçtavsiye...,Olumsuz
4,kulaklığın sesi kaliteli falan değil aleti öve...,Olumsuz


--------------------------------------------------------------

In [369]:
# Import the nltk library and download stopwords
#import nltk

#nltk.download("stopwords")

In [370]:
# Get the stopwords
#from nltk.corpus import stopwords
#
#stop_words = stopwords.words("turkish")
#stop_words[:10]

In [371]:
# Remove stopwords from each line and check the lines
#stop_words = set(stop_words)
#df['Görüş'] = df['Görüş'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
#
#df.Görüş[0]

In [372]:
#!pip install TurkishStemmer

In [373]:
# Import the library and get the stemmer for Turkish Language
#from TurkishStemmer import TurkishStemmer

#stemmer = TurkishStemmer()

In [374]:
# To use stemmer on each word, turn each line into a list
#df['Görüş'] = df['Görüş'].str.split()
#df.head()

In [375]:
# Apply stemmer
#df['Görüş'] = df['Görüş'].apply(lambda x: [stemmer.stem(y) for y in x])

#df.head()

In [376]:
#df.Görüş[0]

In [377]:
# Turn each line back to a string (from list)
#df['Görüş'] = df['Görüş'].apply(lambda x: ' '.join(word for word in x))

#df.head()

-------------------------------------------

In [378]:
# Get train sentences from df.Görüş
train_sentences = df["Görüş"].tolist()

train_sentences[0]

'ses kalitesi ve ergonomisi rezalet sony olduğu için aldım ama 4de 1 fiyatına çin replika ürün alsaydım çok çok daha iyiydi kesinlikle tavsiye etmiyorum'

In [379]:
# Shuffle the data so our model can learn in a proper way

from sklearn.utils import shuffle
df = shuffle(df)
df.head()

,Görüş,Durum
10,çok değil çekim gücü az olduğu için 3 puan,Tarafsız
1522,avantajlar fiyatbr i̇nternette yaptığım araşt...,Olumlu
734,ürün çok güzel paketlenmiş mükemmel kalitede,Olumlu
2504,hayalisukuta uğradım iki aydır araştırıyorum b...,Olumsuz
4523,ürün çin malı menüsü karışık kullanımı çocukla...,Tarafsız


In [380]:
# Check the DataFrame for one last time
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11426 entries, 10 to 9877
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Görüş   11426 non-null  object
 1   Durum   11426 non-null  object
dtypes: object(2)
memory usage: 267.8+ KB


### Input pipeline

In [381]:
# Get the average and the max length of the inputs
import numpy as np

sent_lens = [len(sentence.split()) for sentence in train_sentences]
avg_sent_len = np.mean(sent_lens)
max_sent_len = np.max(sent_lens)
avg_sent_len, max_sent_len

(21.72378785226676, 422)

In [382]:
# How long of a sentence lenght covers 95% of examples?
output_seq_len_95 = int(np.percentile(sent_lens, 95))

output_seq_len_95

64

In [383]:
# How long of a sentence lenght covers 99% of examples?
output_seq_len_99 = int(np.percentile(sent_lens, 99))

output_seq_len_99

120

In [384]:
# Since it is not a large number, 120 is chosen. The GPU that we're using can handle this
output_seq_len = 120

In [385]:
# Get transformers
!pip install transformers

In [386]:
# Import AutoTokenizer
from transformers import AutoTokenizer
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-128k-uncased")

In [387]:
# Encode inputs
import tensorflow as tf

input_ids = []
attention_mask = []

for txt in df.Görüş.values:
    encoded = tokenizer.encode_plus(
        text=txt, # the sentence to be encoded 
        add_special_tokens=True, # Add [CLS] and [SEP]
        max_length=output_seq_len, # max length of a sentence
        truncation=True, # truncate if sentence length is bigger than max_length
        pad_to_max_length=True, # Add [PAD]s
        return_attention_mask=True, # Generate attention mask
        return_tensors="tf" # return TensorFlow tensors
    )

    # Append input_ids and attention_masks to their own lists
    input_ids.append(encoded["input_ids"])
    attention_mask.append(encoded["attention_mask"])

# Concatenate
input_ids = tf.concat(input_ids, 0)
attention_mask = tf.concat(attention_mask, 0)

print("Original: ", df.Görüş.values[0])
print("Token IDs: ", input_ids[0])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  çok değil çekim gücü az olduğu için 3 puan
Token IDs:  tf.Tensor(
[     2   6110  10715 106798  48790   1012   2463  15500   8059     23
   4058      3      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0], shape=(120,), dtype=int32)


In [388]:
# Convert tokens to ids to check if the encoding operation is done correctly
tokenizer.convert_ids_to_tokens(input_ids[0])

['[CLS]',
 'cok',
 'degil',
 'cekim',
 'guc',
 '##u',
 'az',
 'oldugu',
 'icin',
 '3',
 'puan',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[P

In [389]:
# Check input_ids and shape of input_ids
input_ids, input_ids.shape

(<tf.Tensor: shape=(11426, 120), dtype=int32, numpy=
 array([[     2,   6110,  10715, ...,      0,      0,      0],
        [     2,  26965,   2821, ...,      0,      0,      0],
        [     2,  36664,   6110, ...,      0,      0,      0],
        ...,
        [     2,  16294,   2101, ...,      0,      0,      0],
        [     2,   4991,  70375, ...,      0,      0,      0],
        [     2, 106798,  48790, ...,      0,      0,      0]], dtype=int32)>,
 TensorShape([11426, 120]))

In [390]:
# Check attention_mask and shape of attention_mask
attention_mask, attention_mask.shape

(<tf.Tensor: shape=(11426, 120), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>, TensorShape([11426, 120]))

In [391]:
# Seperate test df from original df
test_df = df.iloc[:1100]
df = df.iloc[1100:]
len(df), len(test_df)

(10326, 1100)

In [392]:
# Seperate test input ids and test attention mask from the original ones
test_input_ids = input_ids[:1100]
test_attention_mask = attention_mask[:1100]

input_ids = input_ids[1100:]
attention_mask = attention_mask[1100:]

input_ids.shape, test_input_ids.shape, attention_mask.shape, test_attention_mask.shape

(TensorShape([10326, 120]),
 TensorShape([1100, 120]),
 TensorShape([10326, 120]),
 TensorShape([1100, 120]))

In [393]:
# One hot encode our labels to use in our models
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
labels_one_hot = one_hot_encoder.fit_transform(df["Durum"].to_numpy().reshape(-1,1))
labels_one_hot.shape

(10326, 3)

In [394]:
# One hot encode test labels
one_hot_encoder = OneHotEncoder(sparse=False)
test_labels_one_hot = one_hot_encoder.fit_transform(test_df["Durum"].to_numpy().reshape(-1,1))
test_labels_one_hot.shape

(1100, 3)

In [395]:
# Label encode test labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
test_labels_encoded = label_encoder.fit_transform(test_df["Durum"].to_numpy())

test_labels_encoded

array([2, 0, 0, ..., 2, 2, 1])

In [396]:
# Create a TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices((input_ids, attention_mask, labels_one_hot))
test_dataset = tf.data.Dataset.from_tensor_slices((test_input_ids, test_attention_mask, test_labels_one_hot))
dataset, test_dataset

(<TensorSliceDataset shapes: ((120,), (120,), (3,)), types: (tf.int32, tf.int32, tf.float64)>,
 <TensorSliceDataset shapes: ((120,), (120,), (3,)), types: (tf.int32, tf.int32, tf.float64)>)

In [397]:
# Create a function to map our dataset
def map_func(input_ids, masks, labels):
    # We convert our three-item tuple into a two-item tuple where the input item is a dictionary
    return {"input_ids": input_ids,
            "attention_mask": masks}, labels

In [398]:
# Map the dataset using the function we created and check the dataset
dataset = dataset.map(map_func)
test_dataset = test_dataset.map(map_func)
dataset, test_dataset

(<MapDataset shapes: ({input_ids: (120,), attention_mask: (120,)}, (3,)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>,
 <MapDataset shapes: ({input_ids: (120,), attention_mask: (120,)}, (3,)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>)

In [399]:
# Get the length of our dataset
len_dataset_unbatched = len(dataset)

In [400]:
# Batch our datasets
batch_size=32
dataset = dataset.batch(batch_size) #.shuffle(10000) , drop_remainder=True
test_dataset = test_dataset.batch(batch_size)

dataset.take, test_dataset

(<bound method DatasetV2.take of <BatchDataset shapes: ({input_ids: (None, 120), attention_mask: (None, 120)}, (None, 3)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>>,
 <BatchDataset shapes: ({input_ids: (None, 120), attention_mask: (None, 120)}, (None, 3)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>)

In [401]:
# Split our dataset into train, validation and test datasets
split = 0.85
size = int((input_ids.shape[0] / batch_size) * split)

train_ds = dataset.take(size) # 85% of the dataset
val_ds = dataset.skip(size) # 15% of the dataset

len(dataset), len(train_ds), len(val_ds)

(323, 274, 49)

## Build and train

In [139]:
# Import the model
from transformers import TFAutoModel

bert128k = TFAutoModel.from_pretrained("dbmdz/bert-base-turkish-128k-uncased")

Some layers from the model checkpoint at dbmdz/bert-base-turkish-128k-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [140]:
# Create the model
import tensorflow as tf

# two input layers, we ensure layer name variables match to dictionary keys in TF dataset
input_ids = tf.keras.layers.Input(shape=(120,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(120,), name='attention_mask', dtype='int32')

# we access the transformer model within our bert object using the bert attribute (eg bert.bert instead of bert)
embeddings = bert128k.bert(input_ids, attention_mask=mask)[1]  # access final activations (already max-pooled) [1]
# convert bert embeddings into 3 output classes
x = tf.keras.layers.Dense(1024, activation='relu')(embeddings)
outputs = tf.keras.layers.Dense(3, activation='softmax', name='outputs')(x)

# model
model_1_128k_uncased = tf.keras.Model(inputs=[input_ids, mask], outputs=outputs)

In [141]:
#Get the summary of model_1_128k_uncased
model_1_128k_uncased.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 120)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 120)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  184345344   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 120,                                         

In [142]:
# Get learning rate using PolynomialDecay
from tensorflow.keras.optimizers.schedules import PolynomialDecay

x = len_dataset_unbatched*0.85 # length of train_ds (unbatched)

num_epochs = 3
num_train_steps = x * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5,
    end_learning_rate=0.,
    decay_steps=num_train_steps
)

In [143]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model_1_128k_uncased.compile(optimizer=optimizer, 
              loss=loss, 
              metrics=[acc])

In [144]:
# Fit the model
history = model_1_128k_uncased.fit(
    train_ds,
    validation_data=val_ds,
    epochs=4,
    verbose=1
)

Epoch 1/4
274/274 [==============================] - 431s 1s/step - loss: 0.7413 - accuracy: 0.6658 - val_loss: 0.7084 - val_accuracy: 0.7073
Epoch 2/4
274/274 [==============================] - 407s 1s/step - loss: 0.5633 - accuracy: 0.7709 - val_loss: 0.6632 - val_accuracy: 0.7401
Epoch 3/4
274/274 [==============================] - 407s 1s/step - loss: 0.4469 - accuracy: 0.8254 - val_loss: 0.6962 - val_accuracy: 0.7420
Epoch 4/4
274/274 [==============================] - 407s 1s/step - loss: 0.3309 - accuracy: 0.8809 - val_loss: 0.7932 - val_accuracy: 0.7471


In [145]:
# Evaluate the model on test_ds
model_1_128k_uncased.evaluate(test_dataset)

35/35 [==============================] - 17s 473ms/step - loss: 0.7069 - accuracy: 0.7673


[0.7068949937820435, 0.7672727108001709]

In [146]:
# Download helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-12-07 22:03:43--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-12-07 22:03:43 (75.1 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [147]:
from helper_functions import calculate_results

In [148]:
# Make predictions
model_1_128k_uncased_pred_probs = model_1_128k_uncased.predict(test_dataset)
model_1_128k_uncased_pred_probs[0], model_1_128k_uncased_pred_probs.shape

(array([0.8083878 , 0.0064343 , 0.18517783], dtype=float32), (1100, 3))

In [149]:
# Convert pred_probs to classes
model_1_128k_uncased_preds = tf.argmax(model_1_128k_uncased_pred_probs, axis=1)
model_1_128k_uncased_preds

<tf.Tensor: shape=(1100,), dtype=int64, numpy=array([0, 1, 0, ..., 0, 1, 0])>

In [150]:
# Calculate model_1 results
model_1_results = calculate_results(y_true=test_labels_encoded,
                                    y_pred=model_1_128k_uncased_preds)
model_1_results

{'accuracy': 76.72727272727272,
 'f1': 0.7695097795934159,
 'precision': 0.7722493926925608,
 'recall': 0.7672727272727272}

In [259]:
import tensorflow as tf

# two input layers, we ensure layer name variables match to dictionary keys in TF dataset
input_ids = tf.keras.layers.Input(shape=(120,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(120,), name='attention_mask', dtype='int32')

# we access the transformer model within our bert object using the bert attribute (eg bert.bert instead of bert)
embeddings = bert128k.bert(input_ids, attention_mask=mask)[0]  # access final activations with [0]

x = tf.keras.layers.LSTM(8, dropout=.4, recurrent_dropout=.4, return_sequences=False) (embeddings) 
# normalize
x = tf.keras.layers.BatchNormalization()(x)
# output
outputs = tf.keras.layers.Dense(3, activation='softmax', name='outputs')(x)

model_2_128k_uncased_LSTM = tf.keras.Model(inputs=[input_ids, mask], outputs=outputs)

In [260]:
model_2_128k_uncased_LSTM.layers[2].trainable = False

# print out model summary
model_2_128k_uncased_LSTM.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 120)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 120)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  184345344   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 120,                                         

In [261]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) #lr_scheduler
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model_2_128k_uncased_LSTM.compile(optimizer=optimizer, 
              loss=loss, 
              metrics=[acc])

In [262]:
# Fit the model
history_model_2 = model_2_128k_uncased_LSTM.fit(
    train_ds,
    validation_data=val_ds,
    epochs=4,
    verbose=1
)

Epoch 1/4
274/274 [==============================] - 354s 1s/step - loss: 0.6841 - accuracy: 0.7148 - val_loss: 0.4174 - val_accuracy: 0.8588
Epoch 2/4
274/274 [==============================] - 331s 1s/step - loss: 0.4537 - accuracy: 0.8418 - val_loss: 0.3702 - val_accuracy: 0.8800
Epoch 3/4
274/274 [==============================] - 332s 1s/step - loss: 0.4196 - accuracy: 0.8549 - val_loss: 0.3554 - val_accuracy: 0.8858
Epoch 4/4
274/274 [==============================] - 332s 1s/step - loss: 0.4054 - accuracy: 0.8607 - val_loss: 0.3489 - val_accuracy: 0.8890


In [263]:
# Evaluate the model on test_ds
model_2_128k_uncased_LSTM.evaluate(test_dataset)

35/35 [==============================] - 19s 529ms/step - loss: 0.3291 - accuracy: 0.8873


[0.32908207178115845, 0.8872727155685425]

In [264]:
# Make predictions
model_2_128k_uncased_LSTM_pred_probs = model_2_128k_uncased_LSTM.predict(test_dataset)
model_2_128k_uncased_LSTM_pred_probs[0], model_2_128k_uncased_LSTM_pred_probs.shape

(array([0.00780847, 0.9600811 , 0.03211032], dtype=float32), (1100, 3))

In [265]:
# Convert pred_probs to classes
model_2_128k_uncased_LSTM_preds = tf.argmax(model_2_128k_uncased_LSTM_pred_probs, axis=1)
model_2_128k_uncased_LSTM_preds

<tf.Tensor: shape=(1100,), dtype=int64, numpy=array([1, 0, 1, ..., 2, 2, 1])>

In [266]:
# Calculate model_2 results
model_2_results = calculate_results(y_true=test_labels_encoded,
                                    y_pred=model_2_128k_uncased_LSTM_preds)
model_2_results

{'accuracy': 88.72727272727273,
 'f1': 0.8848936799629704,
 'precision': 0.8851480160049142,
 'recall': 0.8872727272727273}

In [353]:
# two input layers, we ensure layer name variables match to dictionary keys in TF dataset
input_ids = tf.keras.layers.Input(shape=(120,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(120,), name='attention_mask', dtype='int32')

# we access the transformer model within our bert object using the bert attribute (eg bert.bert instead of bert)
embeddings = bert128k.bert(input_ids, attention_mask=mask)[0]  # access final activations with [0]

x = tf.keras.layers.GRU(16, return_sequences=False)(embeddings) 
# normalize
x = tf.keras.layers.BatchNormalization()(x)
# output
outputs = tf.keras.layers.Dense(3, activation='softmax', name='outputs')(x)

model_3_128k_uncased_GRU = tf.keras.Model(inputs=[input_ids, mask], outputs=outputs)

In [354]:
model_3_128k_uncased_GRU.layers[2].trainable = False

# print out model summary
model_3_128k_uncased_GRU.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 120)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 120)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  184345344   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 120,                                         

In [355]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) #lr_scheduler
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model_3_128k_uncased_GRU.compile(optimizer=optimizer, 
              loss=loss, 
              metrics=[acc])

In [356]:
# Fit the model
history_model_3 = model_3_128k_uncased_GRU.fit(
    train_ds,
    validation_data=val_ds,
    epochs=4,
    verbose=1
)

Epoch 1/4
274/274 [==============================] - 190s 626ms/step - loss: 0.6361 - accuracy: 0.7525 - val_loss: 0.3886 - val_accuracy: 0.8716
Epoch 2/4
274/274 [==============================] - 168s 612ms/step - loss: 0.4264 - accuracy: 0.8493 - val_loss: 0.3570 - val_accuracy: 0.8813
Epoch 3/4
274/274 [==============================] - 168s 612ms/step - loss: 0.3964 - accuracy: 0.8629 - val_loss: 0.3443 - val_accuracy: 0.8870
Epoch 4/4
274/274 [==============================] - 168s 612ms/step - loss: 0.3871 - accuracy: 0.8678 - val_loss: 0.3373 - val_accuracy: 0.8902


In [357]:
# Evaluate the model on test_ds
model_3_128k_uncased_GRU.evaluate(test_dataset)

35/35 [==============================] - 17s 478ms/step - loss: 0.3365 - accuracy: 0.8873


[0.3365486264228821, 0.8872727155685425]

In [358]:
# Make predictions
model_3_128k_uncased_GRU_pred_probs = model_3_128k_uncased_GRU.predict(test_dataset)
model_3_128k_uncased_GRU_pred_probs[0], model_3_128k_uncased_GRU_pred_probs.shape

(array([0.01193844, 0.9718598 , 0.01620176], dtype=float32), (1100, 3))

In [359]:
# Convert pred_probs to classes
model_3_128k_uncased_GRU_preds = tf.argmax(model_3_128k_uncased_GRU_pred_probs, axis=1)
model_3_128k_uncased_GRU_preds

<tf.Tensor: shape=(1100,), dtype=int64, numpy=array([1, 1, 0, ..., 2, 1, 0])>

In [360]:
# Calculate model_3 results
model_3_results = calculate_results(y_true=test_labels_encoded,
                                    y_pred=model_3_128k_uncased_GRU_preds)
model_3_results

{'accuracy': 88.72727272727273,
 'f1': 0.8860886106830129,
 'precision': 0.8856997214280882,
 'recall': 0.8872727272727273}

In [402]:
# two input layers, we ensure layer name variables match to dictionary keys in TF dataset
input_ids = tf.keras.layers.Input(shape=(120,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(120,), name='attention_mask', dtype='int32')

# we access the transformer model within our bert object using the bert attribute (eg bert.bert instead of bert)
embeddings = bert128k.bert(input_ids, attention_mask=mask)[1]  # access final activations with [0]

x = tf.keras.layers.Dense(16, activation="relu")(embeddings)
# output
outputs = tf.keras.layers.Dense(3, activation='softmax', name='outputs')(x)

model_4_dense = tf.keras.Model(inputs=[input_ids, mask], outputs=outputs)

In [403]:
model_4_dense.layers[2].trainable = False

# print out model summary
model_4_dense.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 120)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 120)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  184345344   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 120,                                        

In [405]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(lr=3e-5)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model_4_dense.compile(optimizer=optimizer, 
              loss=loss, 
              metrics=[acc])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [406]:
# Fit the model
history_model_4 = model_4_dense.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3,
    verbose=1
)

Epoch 1/3
274/274 [==============================] - 185s 614ms/step - loss: 0.8585 - accuracy: 0.6551 - val_loss: 0.4298 - val_accuracy: 0.8755
Epoch 2/3
274/274 [==============================] - 165s 602ms/step - loss: 0.4014 - accuracy: 0.8625 - val_loss: 0.3698 - val_accuracy: 0.8819
Epoch 3/3
274/274 [==============================] - 165s 601ms/step - loss: 0.3788 - accuracy: 0.8654 - val_loss: 0.3644 - val_accuracy: 0.8851


In [407]:
# Evaluate the model on test_ds
model_4_dense.evaluate(test_dataset)

35/35 [==============================] - 17s 471ms/step - loss: 0.3104 - accuracy: 0.8936


[0.3103732466697693, 0.8936363458633423]

In [408]:
# Make predictions
model_4_dense_pred_probs = model_4_dense.predict(test_dataset)
model_4_dense_pred_probs[0], model_4_dense_pred_probs.shape

(array([0.48290142, 0.1296168 , 0.38748175], dtype=float32), (1100, 3))

In [409]:
# Convert pred_probs to classes
model_4_dense_preds = tf.argmax(model_4_dense_pred_probs, axis=1)
model_4_dense_preds

<tf.Tensor: shape=(1100,), dtype=int64, numpy=array([0, 0, 0, ..., 2, 2, 1])>

In [410]:
# Calculate model_3 results
model_4_results = calculate_results(y_true=test_labels_encoded,
                                    y_pred=model_4_dense_preds)
model_4_results

{'accuracy': 89.36363636363637,
 'f1': 0.8930262630968501,
 'precision': 0.8927291539405786,
 'recall': 0.8936363636363637}